In [ ]:
# import used tools
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from scipy import stats

import time
print(tf.__version__)

In [ ]:
# import the data for dim=4
# column ‘0’ is the target: robustness of coherence
# the rest columns are the matrix elements
# the filenames can be renamed in the matlab program which generates the data


# the following 2 files are the data for mixed states
dataset_path = 'realEntries4.csv'
dataset_path_1 = 'realEntries4-1.csv'
dataset_path_pure = 'realEntriesPure4.csv' # pure state

# combining all the data into a single dataset
column_names=list(range(17))
column_names = [str(x) for x in column_names]
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      skipinitialspace=True)
raw_dataset_1 = pd.read_csv(dataset_path_1, names=column_names,
                      na_values = "?", comment='\t',
                      skipinitialspace=True)

raw_dataset_pure = pd.read_csv(dataset_path_pure, names=column_names,
                      na_values = "?", comment='\t',
                     skipinitialspace=True)


# concate into a single dataset
raw_dataset = pd.concat([raw_dataset, raw_dataset_1, raw_dataset_pure], ignore_index=True)


dataset = raw_dataset.copy()
dataset.head()
dataset.shape

In [ ]:
# To check for bad values in the data and remove those rows.

dataset.isna().sum()
dataset = dataset.dropna()

In [ ]:
# To split the data into a training set and a test set.

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# To check the size of the data.
test_dataset.shape


In [ ]:
# To inspect the elements of the matrices in the training set.
train_stats = train_dataset.describe()
train_stats.pop("0")
train_stats = train_stats.transpose()
train_stats

In [ ]:
# To separate the target from the dataset.
train_labels = train_dataset.pop('0')
test_labels = test_dataset.pop('0')

In [ ]:
# To normalize the data.
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset).fillna(0)
normed_test_data = norm(test_dataset).fillna(0)

In [ ]:
# To build the model.
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),  
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  #optimizer = tf.keras.optimizers.RMSprop(0.0001)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  
  plt.style.use('seaborn')  
  plt.figure(figsize=(4,3),dpi=120)
  plt.xlabel('Epoch')
  plt.ylabel('MAE')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error',color='#00468C')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Validation Error',color='#E95C4B')
  plt.ylim([0,0.05])
  plt.legend()
  plt.savefig('d4MAE.eps',bbox_inches='tight')
  plt.figure(figsize=(4,3),dpi=120)

  plt.xlabel('Epoch')
  plt.ylabel('MSE')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error',color='#00468C')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Validation Error',color='#E95C4B')
  plt.xlim([-0.2,45])
  plt.ylim([0,0.001])
  plt.legend()
  plt.savefig('d4MSE.eps',bbox_inches='tight')
  plt.show()

In [ ]:
# To train the model

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')
    
EPOCHS = 1000

model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
# To check the condition of the test set.

loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))

In [ ]:
# check the performance of prediction

test_predictions = model.predict(normed_test_data).flatten()

r2 = round(1 - sum((test_labels - test_predictions) ** 2)/sum((test_labels - np.mean(test_labels)) ** 2),4)

plt.figure(figsize=(4,4),dpi=120)
plt.style.use('classic')
plt.text(1.5,0.75, "d = 4",fontsize=16, fontweight='bold', style = 'italic', fontfamily='Times New Roman')
plt.text(1.5,0.5,"R$^2$ = " + str(r2) + "",fontsize=16, fontweight='bold', style = 'italic', fontfamily='Times New Roman')
plt.scatter(test_labels[:5000], test_predictions[:5000],
            s=20,c='none',edgecolor=(20/255,81/255,124/255),marker="o",alpha=1,linewidth=0.3)
plt.xlabel('Actual ROC')
plt.ylabel('Predicted ROC')
plt.axis('equal')
plt.axis('square')
plt.xlim([-0.,3.])
plt.ylim([-0.,3.])
plt.plot([0, 10.05], [0, 10],c='r',lw=1.5,alpha=1)
plt.xticks(fontsize=12, fontname='Times New Roman')
plt.yticks(fontsize=12, fontname='Times New Roman')


plt.savefig('d4.eps',bbox_inches='tight')
plt.show()

In [ ]:
# save the model
model.save('d4.h5')

In [ ]:
# load the model
model = tf.keras.models.load_model('d4.h5')

In [ ]:
# import the non_markov data
# column ‘0’ is the target: robustness of coherence
# the rest columns are the matrix elements

dataset_path = 'nonmarkovPhi.csv'

#nonmarkov

column_names=list(range(17))
column_names = [str(x) for x in column_names]
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.head()




In [ ]:
# clean the data
dataset.isna().sum()
dataset = dataset.dropna()

labels = dataset.pop('0')



In [ ]:
# normalize the data
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_data = norm(dataset)

normed_data=normed_data.fillna(0)

In [ ]:
# check the performance of prediction


predictions = model.predict(normed_data)

plt.figure(figsize=(4,3), dpi=130)
plt.style.use('classic')

plt.plot(predictions, linestyle='-',c = (20/255,81/255,124/255), marker="o",alpha=1, markeredgecolor= (20/255,81/255,124/255), markerfacecolor = 'none',markeredgewidth = 1, label='Predicted ROC')
plt.plot(labels,'-', color='red', label= 'Actual ROC')
plt.xlabel('$\Gamma$t')
plt.ylabel('ROC')
plt.ylim([0,1])
plt.xticks(fontsize=12, fontname='Times New Roman')  
plt.yticks(fontsize=12, fontname='Times New Roman')  
plt.legend()
plt.savefig('nonmarkovphi.eps',bbox_inches='tight')

In [ ]:
# save the model

model.save('d4.keras')

In [ ]:
# load the model

model = tf.keras.models.load_model('d4.keras')